# Projeto - Pegando dados de ETFs do mundo inteiro com Python utilizando Web Scraping.

In [17]:
!pip install webdriver-manager

In [18]:
!pip install selenium

In [20]:
!pip install html5lib

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Conhecer e mapear o processo de coleta de dados no site do ETF.com:

In [68]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-finder"

driver.get(url)

# Coletando todos os elementos necessários dentro do HTML do site - Expandindo a tabela para 100 itens:

In [26]:
time.sleep(5)

botao_100 = driver.find_element("xpath", '''/html/body/div[4]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", botao_100)

# Buscando todos os elementos necessários dentro do HTML do site - Pegando o número de páginas da tabela:

In [27]:
numero_paginas = driver.find_element("xpath", '''/html/body/div[4]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]''')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

print(numero_paginas)

31


# Lendo a tabela de dados básicos:

In [67]:
lista_de_tabela_por_pagina = []

numero_paginas = 31

for pagina in range(0, numero_paginas):
    try:
        tabela = driver.find_element("xpath", '''/html/body/div[4]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div/table''')
        
        html_tabela = tabela.get_attribute("outerHTML")
        
        tabela_final = pd.read_html(html_tabela)[0]
        
        lista_de_tabela_por_pagina.append(tabela_final)
        
        botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
        
        driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    except:
        # In case an element is not found or an error occurs, the loop should continue instead of stopping
        print(f"Error on page {pagina + 1}. Continuing to next page.")
        continue

base_de_dados_completa = pd.concat(lista_de_tabela_por_pagina)

display(base_de_dados_completa)

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,KGRO,KraneShares China Innovation ETF,Equity: China Broad Thematic,CICC,0.64%,$1.67M
1,SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.64M
2,BULD,Pacer BlueStar Engineering the Future ETF,Equity: Global Robotics & AI,Pacer Advisors,0.60%,$1.63M
3,TINT,ProShares Smart Materials ETF,Equity: Global Materials,ProShares,0.58%,$1.63M
4,STLG,iShares Factors U.S. Growth Style ETF,Equity: U.S. - Large Cap Growth,Blackrock,0.25%,$1.61M
...,...,...,...,...,...,...
85,AWYX,ETFMG 2x Daily Travel Tech ETF,Leveraged Equity: Global Internet & Direct Mar...,ETFMG,0.95%,$338.71K
86,TADS,The Active Dividend Stock ETF,Equity: U.S. - Total Market,"Tuttle Tactical Management, LLC",1.68%,$294.89K
87,CRYP,AdvisorShares Managed Bitcoin Strategy ETF,Asset Allocation: Global Target Outcome,AdvisorShares,1.59%,$216.78K
88,FLRU,Franklin FTSE Russia ETF,Equity: Russia - Total Market,Franklin Templeton,0.19%,$8.00K


# Lendo a tabela de dados de rentabilidade e performance:

In [81]:
botao_aba = driver.find_element("xpath", '''/html/body/div[4]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div/table''')

driver.execute_script("arguments[0].click();", botao_aba)

for pagina in range(0, numero_paginas):
    
    botao_voltar_pagina = driver.find_element("xpath", '//*[@id="previousPage"]')
    
    driver.execute_script("arguments[0].click();", botao_voltar_pagina)

In [82]:
lista_de_tabela_por_pagina = []

numero_paginas = 31

for pagina in range(0, numero_paginas):
    try:
        tabela = driver.find_element("xpath", '''/html/body/div[4]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div/table''')
        
        html_tabela = tabela.get_attribute("outerHTML")
        
        tabela_final = pd.read_html(html_tabela)[0]
        
        lista_de_tabela_por_pagina.append(tabela_final)
        
        botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
        
        driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    except:
        # In case an element is not found or an error occurs, the loop should continue instead of stopping
        print(f"Error on page {pagina + 1}. Continuing to next page.")
        continue

base_de_dados_performance = pd.concat(lista_de_tabela_por_pagina)

display(base_de_dados_performance)

,Ticker,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
0,XHYH,BondBloxx USD High Yield Bond Healthcare Secto...,3.23%,8.63%,3.96%,--,--,--,--,01/20/23
1,AGGH,Simplify Aggregate Bond PLUS Credit Hedge ETF,0.50%,5.03%,1.83%,--,--,--,--,01/20/23
2,KGRO,KraneShares China Innovation ETF,16.09%,33.28%,13.33%,-18.54%,--,--,--,01/20/23
3,TINT,ProShares Smart Materials ETF,9.29%,19.28%,9.99%,-17.69%,--,--,--,01/20/23
4,BULD,Pacer BlueStar Engineering the Future ETF,9.31%,21.14%,10.29%,--,--,--,--,01/20/23
...,...,...,...,...,...,...,...,...,...,...
85,AWYX,ETFMG 2x Daily Travel Tech ETF,36.11%,38.66%,31.27%,-48.46%,--,--,--,01/20/23
86,TADS,The Active Dividend Stock ETF,0%,0%,0%,0%,--,--,--,01/20/23
87,CRYP,AdvisorShares Managed Bitcoin Strategy ETF,--,--,--,--,--,--,--,01/20/23
88,FLRU,Franklin FTSE Russia ETF,0%,0%,0%,-66.37%,-31.18%,--,--,01/20/23


# Construindo a tabela final: Juntando os dados das duas tabelas:

In [83]:
base_de_dados_completa = base_de_dados_completa.set_index("Ticker")

base_de_dados_completa

,Name,Segment,Issuer,Expense Ratio,AUM
Ticker,,,,,
KGRO,KraneShares China Innovation ETF,Equity: China Broad Thematic,CICC,0.64%,$1.67M
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.64M
BULD,Pacer BlueStar Engineering the Future ETF,Equity: Global Robotics & AI,Pacer Advisors,0.60%,$1.63M
TINT,ProShares Smart Materials ETF,Equity: Global Materials,ProShares,0.58%,$1.63M
STLG,iShares Factors U.S. Growth Style ETF,Equity: U.S. - Large Cap Growth,Blackrock,0.25%,$1.61M
...,...,...,...,...,...
AWYX,ETFMG 2x Daily Travel Tech ETF,Leveraged Equity: Global Internet & Direct Mar...,ETFMG,0.95%,$338.71K
TADS,The Active Dividend Stock ETF,Equity: U.S. - Total Market,"Tuttle Tactical Management, LLC",1.68%,$294.89K
CRYP,AdvisorShares Managed Bitcoin Strategy ETF,Asset Allocation: Global Target Outcome,AdvisorShares,1.59%,$216.78K


In [85]:
base_de_dados_performance = base_de_dados_performance.set_index("Ticker")

base_de_dados_performance = base_de_dados_performance[['1 Year', '5 Years', '10 Years']]

base_de_dados_performance

,1 Year,5 Years,10 Years
Ticker,,,
XHYH,--,--,--
AGGH,--,--,--
KGRO,-18.54%,--,--
TINT,-17.69%,--,--
BULD,--,--,--
...,...,...,...
AWYX,-48.46%,--,--
TADS,0%,--,--
CRYP,--,--,--


In [87]:
base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance)

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years
Ticker,,,,,,,,
AAPB,GraniteShares 1.75x Long AAPL Daily ETF,Leveraged Equity: U.S. Technology Hardware Sto...,GraniteShares,1.15%,$997.02K,--,--,--
AAPB,GraniteShares 1.75x Long AAPL Daily ETF,Leveraged Equity: U.S. Technology Hardware Sto...,GraniteShares,1.15%,$997.02K,--,--,--
AAPB,GraniteShares 1.75x Long AAPL Daily ETF,Leveraged Equity: U.S. Technology Hardware Sto...,GraniteShares,1.15%,$997.02K,--,--,--
AAPB,GraniteShares 1.75x Long AAPL Daily ETF,Leveraged Equity: U.S. Technology Hardware Sto...,GraniteShares,1.15%,$997.02K,--,--,--
AAPB,GraniteShares 1.75x Long AAPL Daily ETF,Leveraged Equity: U.S. Technology Hardware Sto...,GraniteShares,1.15%,$997.02K,--,--,--
...,...,...,...,...,...,...,...,...
ZGEN,EA Series Trust - The Generation Z ETF,Equity: Global Consumer,Alpha Architect,0.63%,$1.42M,-42.70%,--,--
ZGEN,EA Series Trust - The Generation Z ETF,Equity: Global Consumer,Alpha Architect,0.63%,$1.42M,-42.70%,--,--
ZGEN,EA Series Trust - The Generation Z ETF,Equity: Global Consumer,Alpha Architect,0.63%,$1.42M,-42.70%,--,--
